In [8]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import csv
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
# embedding_size: the length of the hidden vector for both user and item
emdedding_size_mlp = 20

# batch_size: the number of cases put into our model for one term
batch_size = 100

# num_epoch: the total round we train our model
num_epoch = 100

# learing_rate: the variable chaging speed for our optimizer
learning_rate = 0.001


In [18]:
header = ['user_id', 'item_id', 'rating', 'timestamp']

with open('ratings.csv') as file:
    data = []
    count = 0
    csv_file = csv.reader(file)
    for line in csv_file:
        if count ==0:
            count+= 1
            continue
        else:
            data.append(list(map(float,line)))
    data = np.array(data)

n_users = int(max(data[:,0]))
n_items = int(max(data[:,1]))
# split our whole data into train_data and test_data。
train_data, test_data = train_test_split(data, test_size = 0.1)

Model Definition

In [13]:
# we define our placeholders as the entrance of our data.
# keep_prob is for the drop_out in the full_connect layer.
user_id = tf.placeholder(tf.int32,[batch_size])
item_id = tf.placeholder(tf.int32,[batch_size])
rate = tf.placeholder(tf.float32,[batch_size,1])
keep_prob = tf.placeholder(tf.float32)


# We initialize our mlp user and item embedding matrix randomly here.
# As our user id sart from 1 so the shape is [n_users + 1, emdedding_size], the same for items.
user_embedding_mlp = tf.Variable(tf.random_uniform([n_users + 1, emdedding_size_mlp],0,0.3),trainable = True)
item_embedding_mlp = tf.Variable(tf.random_uniform([n_items + 1, emdedding_size_mlp],0,0.3),trainable = True)

#---------------------MLP Part-------------------#

# We use embedding_lookup to find out the item and user we are training or testing now for the mlp layer,
# both of them are with shape [bath_size, embedding_size_mlp].
user_input_mlp = tf.nn.embedding_lookup(user_embedding_mlp, user_id)
item_input_mlp = tf.nn.embedding_lookup(item_embedding_mlp, item_id)

# Here is the regularization term using
regularizer = tf.contrib.layers.l2_regularizer(0.2)
reg_term = tf.contrib.layers.apply_regularization(regularizer,[user_input_mlp,item_input_mlp])

# We multiply the user hidden layer with the item hidden layer to get their combination with shape [bath_size, embedding_size_mlp]
fc_input = tf.concat([user_input_mlp,item_input_mlp],axis=1)

# Then we put the combined vector into our fc layers, with an out put fc_output.
# The shape of the fc_output is [batch_size, 1], which indicates the nonlinear connection between the item and the user.
w_1 = tf.Variable(tf.truncated_normal([2 * emdedding_size_mlp, 5], stddev=0.1))
b_1 = tf.Variable(tf.constant(0., shape=[5]))
l_1 = tf.nn.tanh(tf.nn.xw_plus_b(fc_input, w_1, b_1))
l_1_drop = tf.nn.dropout(l_1, keep_prob)


w_2 = tf.Variable(tf.truncated_normal([5, 1], stddev=0.1))
b_2 = tf.Variable(tf.constant(0., shape=[1]))
prediction = tf.nn.xw_plus_b(l_1_drop,w_2,b_2)


# We use mean_squared_error as the loss, and plus the regularization term.
loss = tf.losses.mean_squared_error(rate, prediction) + reg_term
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)


W0808 14:11:51.631792 4765603264 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0808 14:11:51.722255 4765603264 deprecation.py:506] From <ipython-input-13-fe5860ceb47f>:33: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0808 14:11:51.845465 4765603264 deprecation.py:323] From /Users/luyaoye/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated a

In [14]:
with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print('Start learning...')
        for epoch in range(15):
            print('epoch: ',format(epoch))
            train_loss = []
            for start,end in zip(range(0,len(train_data), batch_size),range(batch_size,len(train_data),batch_size)):
                tr_loss,_ = sess.run([loss, train], feed_dict={user_id : train_data[start:end,0], item_id : train_data[start:end,1], rate: train_data[start:end,2].reshape(batch_size,1), keep_prob: 0.5})
                train_loss.append(tr_loss)


            rmse = []
            for start, end in zip(range(0, len(test_data), batch_size),range(batch_size, len(test_data), batch_size)):
                pred = sess.run(prediction, feed_dict={user_id : test_data[start:end,0], item_id : test_data[start:end,1], rate: test_data[start:end,2].reshape(batch_size,1), keep_prob: 1.0})
                pred = [[min(max(i[0],1),5)] for i in pred]
                pred = np.array(pred)
                rmse.append((pred - test_data[start:end,2].reshape(batch_size,1))**2)

            print("Train_rmse: {:.3f}, Test_rmse: {:.3f}".format(np.sqrt(np.mean(train_loss)), np.sqrt(np.mean(rmse))))


Start learning...
epoch:  0
Train_rmse: 3.047, Test_rmse: 1.200
epoch:  1
Train_rmse: 1.878, Test_rmse: 1.100
epoch:  2
Train_rmse: 1.561, Test_rmse: 1.026
epoch:  3
Train_rmse: 1.345, Test_rmse: 0.969
epoch:  4
Train_rmse: 1.178, Test_rmse: 0.920
epoch:  5
Train_rmse: 1.063, Test_rmse: 0.900
epoch:  6
Train_rmse: 0.992, Test_rmse: 0.895
epoch:  7
Train_rmse: 0.956, Test_rmse: 0.893
epoch:  8
Train_rmse: 0.936, Test_rmse: 0.894
epoch:  9
Train_rmse: 0.925, Test_rmse: 0.893
epoch:  10
Train_rmse: 0.918, Test_rmse: 0.892
epoch:  11
Train_rmse: 0.913, Test_rmse: 0.892
epoch:  12
Train_rmse: 0.910, Test_rmse: 0.892
epoch:  13
Train_rmse: 0.907, Test_rmse: 0.892
epoch:  14
Train_rmse: 0.905, Test_rmse: 0.895
